# Asignment 1b Task 3

Assignment 1 Question 2 of stanford course, Tensorflow for Deep Learning Research. 
Assignment available here https://web.stanford.edu/class/cs20si/2017/assignments/a1.pdf

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import xlrd
import math
import time

# Question 2 Task 3
 Build a logistic regression model to predict whether someone has coronary heart disease
In the data folder on the class’s GitHub repo, you’ll find the file heart.txt. The first row is the
name of the observed variables. There are 10 variables:
- sbp: Systolic blood pressure
- tobacco: Cumulative tobacco consumption, in kg
- ldl: Low-density lipoprotein cholesterol
- adiposity: Adipose tissue concentration
- famhist: Family history of heart disease (1=Present, 0=Absent)
- typea: Score on test designed to measure type-A behavior
- obesity: Obesity
- alcohol: Current consumption of alcohol
- age: Age of subject
- chd: Coronary heart disease at baseline; 1=Yes 0=No

Each following row contains the information of one patient. There are 462 samples in total.
We will be using the first 9 variables to predict the last variable. That is, your input will be 1-d
tensor of 9 elements, and your label is binary. You should write the function to read in data
yourself, and you should take care of dividing your data into train set and test set.
Report your results and your hyperparameters.
Source of the data:
http://statweb.stanford.edu/~tibs/ElemStatLearn/

In [2]:
from numpy import genfromtxt
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn import preprocessing
import numpy as np


### Import Data

In [3]:
my_data = genfromtxt('heart.csv', delimiter=',')
data_np = my_data[1:463,:]

In [4]:


# Shuffle and divide into train and test
data_np = shuffle(data_np)
data_np = np.asarray(data_np)

X_train = data_np[47:-1,0:9].astype(float)
Y_train = data_np[47:-1,9]
Y_train = Y_train.reshape(-1,1)#.astype(float)
X_test = data_np [0:47,0:9].astype(float)
Y_test = data_np[0:47,9]
Y_test=Y_test.reshape(-1,1)#.astype(float)


In [5]:
print(X_train.shape)

(414, 9)


## Logistic Regression 

In [25]:
# Defing Model
# Define parameters 
tf.reset_default_graph()
learning_rate = 0.001
n_epochs = 10000
lamda = 0.1
LRHeart = tf.Graph()
with LRHeart.as_default():
    #Step 2: 
    X = tf.placeholder(tf.float32,[None,9],name='X')
    Y = tf.placeholder(tf.int32,[None,1],name='Y')
    #Step 3:
    weights = tf.Variable(tf.random_normal(shape=[9,1],stddev=0.001),name='weights') 
    bias = tf.Variable(tf.zeros([1]),name='bias')

    #Step 4:
    logits = tf.matmul(X,weights) + bias
    
    #Step 5: 
    entropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y,logits=logits,name='loss')
    loss = tf.reduce_mean(entropy)+ lamda*tf.norm(weights,ord=2)
    # Step 6:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)  
    # Test 
    probs = tf.nn.softmax(logits)

In [26]:
# Parameters and variables
n_train = X_train.shape[0]
n_test = X_test.shape[0]

# Run Session
with tf.Session(graph=LRHeart) as sess:
    #writer = tf.summary.FileWriter('./graphs/assignment1_b',sess.graph)
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    for i in range(n_epochs):
        _,loss_1 = sess.run([optimizer,loss],feed_dict={X:X_train,Y:Y_train})

        # Prints one hash after 10 batches
    print('Average loss epoch {0}:{1}'.format(i,loss_1/n_train))    
    print('Total_time:{0} seconds',format(time.time()-start_time))
    
# Test The model
    probs_1 = sess.run(probs,feed_dict={X:X_test})
    accuracy_batch = tf.equal(probs_1,Y_test)
    correct_preds_batch= tf.reduce_sum(tf.cast(accuracy_batch,tf.float32))
    accuracy=sess.run(correct_preds_batch)   
print('\nAccuracy:{0}'.format(accuracy/n_test))  
#writer.close()           

Average loss epoch 9999:0.24593539399225356
Total_time:{0} seconds 5.269111156463623

Accuracy:0.3617021276595745


## Logistic Regression Sickit Learn Library

In [31]:
from sklearn import linear_model
from sklearn import metrics
LR = linear_model.LogisticRegression(max_iter=1000)
LR.fit(X_train,Y_train)
y_pred = LR.predict(X_test)
accuracy = metrics.accuracy_score(Y_test, y_pred, sample_weight=None)
print(accuracy)

0.723404255319


C:\Users\ss\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


<span style="color:red">Accuracy with tensorflow(40.43) is way less than scikit learn(68.09). </span>

## Grid Search for Better Logistic Model in tf

In [15]:
run_var=1

In [26]:
# Defing Model
# Define parameters 
tf.reset_default_graph()
#learning_rate = 0.01
n_epochs = 1000
#lamda = 0.001
initializer = tf.contrib.layers.xavier_initializer()

def LogisticRegression(loss_type,lamda,learning_rate,Optimizer=tf.train.GradientDescentOptimizer):
    tf.reset_default_graph()
    
    X = tf.placeholder(tf.float32,[None,9],name='X')
    Y = tf.placeholder(tf.float32,[None,1],name='Y')
    #:
    with tf.name_scope('weight'):
        weights = tf.Variable(initializer(shape=[9,1]),name='weights')
        tf.summary.histogram("weights", weights)
       
    with tf.name_scope('bias'):
        bias = tf.Variable(tf.zeros([1]),name='bias')
        tf.summary.histogram("biases", bias)

        #Step 4:
    with tf.name_scope('logit'):
        logits = tf.matmul(X,weights) + bias 
        tf.summary.histogram("activations", logits)

    #Step 5: 
    with tf.name_scope('entropy'):
        entropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y,logits=logits,name='loss')
        tf.summary.scalar('entropy',entropy)
        loss = tf.reduce_mean(entropy)    
        tf.summary.scalar('loss',loss)
        
        # Step 6:
    with tf.name_scope('Train'):
        optimizer = Optimizer(learning_rate).minimize(loss)  
        # Test 
    with tf.name_scope('Accuracy'):
        # Accuracy
        probs = tf.nn.softmax(logits)
        accuracy_batch = tf.equal(probs,Y)
        accuracy= tf.reduce_mean(tf.cast(accuracy_batch,tf.float32))
        tf.summary.scalar('accuracy',accuracy)
    summary_op = tf.summary.merge_all()
    validation_acc = tf.summary.scalar("Validation_Accuracy",accuracy)     
    #return optimizer,loss
    #to save the graph
    #writer = tf.summary.FileWriter('./graphs/TensorboardTUT/',sess.graph)
    sess = tf.Session()
    writer = tf.summary.FileWriter('%s/%s' % (log_dir, run_var), sess.graph)
    #start_time = time.time()
    sess.run(tf.global_variables_initializer())    
    #Training 
    accuracy_test_np=0
    for epoch in range(n_epochs):
        # Passing through each batch
        print(epoch,end='')
        _,loss_batch,summary= sess.run([optimizer,loss,summary_op],feed_dict={X:X_train,Y:Y_train}) #,summary ,summary_op
        writer.add_summary(summary, epoch )
        #summary_validation = sess.run(validation_acc,feed_dict={X:X_test,Y:Y_test})
        #writer.add_summary(summary_validation,epoch)
        #accuracy_test = sess.run(accuracy,feed_dict={X:X_test,Y:Y_test})
        #accuracy_test_np =accuracy_test 
        print('Average loss epoch {0}:{1}'.format(epoch,loss_batch,accuracy_test_np))          
    writer.close()    

In [27]:
# Parameters and variables
n_train = X_train.shape[0]
n_test = X_test.shape[0]
log_dir = './graphs/Assignment1/Question2c/try2/'
# Run Session
LogisticRegression(loss_type='DRAC1',lamda=0.1,learning_rate=0.01,Optimizer=tf.train.GradientDescentOptimizer)
#with tf.Session(graph=graph1) as sess:
#    sess.run(tf.global_variables_initializer())  
#    _,loss_batch,summary = sess.run([optimizer,loss],feed_dict={X:X_train,Y:Y_train})



0

InvalidArgumentError: tags and values not the same shape: [] != [414] (tag 'entropy/entropy')
	 [[Node: entropy/entropy = ScalarSummary[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](entropy/entropy/tags, entropy/Reshape_2)]]

Caused by op 'entropy/entropy', defined at:
  File "C:\Users\ss\AppData\Local\conda\conda\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\ss\AppData\Local\conda\conda\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\ss\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\ss\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\ss\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOL

In [144]:
# Run Session
log_dir = './graphs/Assignment1/Question2c/try1/'
tf.reset_default_graph()
with tf.Session(graph=graph1) as sess:
    # to save the graph
    #writer = tf.summary.FileWriter('./graphs/TensorboardTUT/',sess.graph)
    writer = tf.summary.FileWriter('%s/%s' % (log_dir, run_var), sess.graph)
    #start_time = time.time()
    sess.run(tf.global_variables_initializer())    
    #Training 
    for epoch in range(n_epochs):
        # Passing through each batch
        _,loss_batch,summary = sess.run([optimizer,loss,summary_op],feed_dict={X:X_train,Y:Y_train})
        writer.add_summary(summary, epoch )
        summary_validatioin = sess.run(validation_acc)
        #writer.add_summary(summary_validation,epoch)
        #print('Average loss epoch {0}:{1}'.format(epoch,loss_batch)        
    
    writer.close()       

NameError: name 'graph1' is not defined

In [ ]:
# Different loss functions    
        if (loss_type=='simple'):
            with tf.name_scope('loss'):
                loss = tf.reduce_mean(entropy)
        elif (loss_type=='l1'):
            with tf.name_scope('loss'):
                loss = tf.reduce_mean(entropy)+ lamda*tf.norm(weights,ord=1)
        elif (loss_type=='l2'):
            with tf.name_scope('loss'):
                loss = tf.reduce_mean(entropy)+ lamda*tf.norm(weights,ord=2)    
        elif (loss_type=='linf'):
            with tf.name_scope('loss'):
                loss = tf.reduce_mean(entropy)+ lamda*tf.norm(weights,ord=np.inf)
        elif (loss_type=='DRAC1'):
            with tf.name_scope('loss'):
                loss = tf.reduce_mean(entropy)+ lamda*tf.norm(weights,ord=np.inf)         
        else: 
            print('Revise loss_type, only accepts: simple,l1,l2,linf,DRAC1')